In [4]:
#Importamos la librerías y variables necesarias

import mysql.connector
import pandas as pd
from flask import Flask, jsonify, request
import plotly as plt
import os
import pickle
import plotly.graph_objects as go
import plotly

cnx = mysql.connector.connect(
    user="admin",
    password="admin123",
    host="test-db.cze2nnbbx5pc.eu-west-3.rds.amazonaws.com",
    database="prueba"
)
cursor = cnx.cursor()

#Definimos una función mediante la que ejecutar las querys devolviendo un dataframe
def make_query_dataframe(code):
    cursor.execute(code)
    results = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]  # Obtener los nombres de las columnas
    df = pd.DataFrame(results, columns=column_names)  # Crear el DataFrame
    return df
def make_query(code):
    cursor.execute(code)
    results = cursor.fetchall()
    return(results)

def predict_all():
    table = "current_employees"
    query = f"SELECT * FROM {table}"
    db = make_query_dataframe(query)
    model = pickle.load(open('JP_0606_1_Ridge.pickle','rb')) # Cambiar ruta modelo
    colums_to_drop = ["id_employee", "name", "involvement", "performance", "environment", "satisfaction", "life_balance", "attrition", "travel", "department",
                     "education", "education_field", "gender", "role", "marital_status", "hours", "department", "years_company"]
    db.drop(columns=colums_to_drop, inplace=True)
    prediction = model.predict(db)
    make_query("DELETE FROM predictions")
    return(prediction)
    query = "UPDATE predictions SET total_months_in_company = %s"
    cursor.executemany(query, [(value,) for value in prediction])
    return prediction

In [6]:
df_risk=make_query_dataframe("""SELECT risk from replacement""")

In [7]:
df_risk.value_counts()

risk     
High         951
Very high    931
Low          918
Medium       898
dtype: int64

In [13]:
import plotly.graph_objects as go

count_values = df_risk.value_counts()

colors = {
    "Low": "#00CC96",
    "Medium": "#B6E880",
    "High": "#FFA15A",
    "Very high": "#EF553B"
}
count_values = {
    "High": 951,
    "Very high": 931,
    "Low": 918,
    "Medium": 898
}
fig = go.Figure(data=[
    go.Pie(
        labels=list(count_values.keys()),  # Convert dict_keys to a list
        values=list(count_values.values()),  # Convert dict_values to a list
        hole=0.4,
        pull=[0.05, 0.05, 0.05, 0.05],
        marker=dict(colors=[colors[label] for label in count_values.keys()])
    )
])
fig.update_traces(textfont=dict(size=22))
fig.update_layout(
    title="Distribution risk attrition",
    title_font=dict(size=24),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5
    ),
    title_x=0.5
)
fig.show()


In [10]:
predict_all()

c:\Users\mgilj\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but Ridge was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


KeyboardInterrupt: 

In [ ]:
# Borrar todos los datos de la tabla
delete_query = "DELETE FROM tu_tabla"
make_query(delete_query)

# Insertar una lista en la columna "date"
data_list = ["2023-06-01", "2023-06-02", "2023-06-03"]
insert_query = "INSERT INTO tu_tabla (date) VALUES (%s)"
for date in data_list:
    make_query(insert_query, (date,))

In [8]:
predict_all()

c:\Users\mgilj\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but Ridge was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


array([-369256.7875975 , -544008.24558491, -233591.69571492, ...,
        -74966.61635119, -103856.65879915,  -66750.3078169 ])